IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M2\FAULT_M2_10


In [3]:
guasto = 2

VOLO m2 FAULT 10%

In [4]:
rcou_m2_fault10 = pd.read_csv("RCOU.csv")

rcou_m2_fault10 = rcou_m2_fault10.astype("float64")
rcou_m2_fault10 = rcou_m2_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m2_fault10 = rcou_m2_fault10[((rcou_m2_fault10['C9'] >= potenza) & (rcou_m2_fault10['C10'] >= potenza) & (rcou_m2_fault10['C11'] >= potenza) & (rcou_m2_fault10['C12'] >= potenza) & (rcou_m2_fault10['C13'] >= potenza) & (rcou_m2_fault10['C14']>= potenza))]
rcou_m2_fault10=rcou_m2_fault10.reset_index(drop=True)
display(rcou_m2_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76591680.0,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0
1,76688094.0,1360.0,1538.0,1465.0,1447.0,1371.0,1530.0
2,76708255.0,1418.0,1514.0,1474.0,1462.0,1423.0,1510.0
3,76739385.0,1527.0,1442.0,1476.0,1497.0,1516.0,1455.0
4,76749554.0,1507.0,1477.0,1499.0,1485.0,1513.0,1471.0
...,...,...,...,...,...,...,...
7760,182518531.0,1516.0,1424.0,1477.0,1466.0,1372.0,1557.0
7761,182575566.0,1449.0,1493.0,1536.0,1399.0,1368.0,1560.0
7762,182618295.0,1471.0,1457.0,1527.0,1392.0,1392.0,1527.0
7763,182682604.0,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0


In [5]:
min=rcou_m2_fault10['TimeUS'][0]
max=rcou_m2_fault10['TimeUS'][len(rcou_m2_fault10)-1]
print(max)
print(min)

182693145.0
76591680.0


In [6]:
xkf1_m2_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m2_fault10 = xkf1_m2_fault10.astype("float64")
xkf1_m2_fault10 = xkf1_m2_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m2_fault10 = xkf1_m2_fault10[((xkf1_m2_fault10['TimeUS'] >= min) & (xkf1_m2_fault10['TimeUS'] <= max	))]
xkf1_m2_fault10 = xkf1_m2_fault10.reset_index(drop=True)
print(xkf1_m2_fault10)

           TimeUS  Roll  Pitch   Yaw
0      76600505.0 -0.11   3.26  3.62
1      76610567.0  0.25   3.25  3.58
2      76620889.0 -0.02   3.32  3.55
3      76630882.0 -0.35   3.39  3.58
4      76641090.0 -0.04   3.35  3.60
...           ...   ...    ...   ...
9638  182649204.0 -0.61  -0.99  0.82
9639  182659430.0 -0.54  -0.98  0.82
9640  182669677.0 -0.47  -0.99  0.83
9641  182680163.0 -0.43  -1.00  0.82
9642  182690890.0 -0.43  -1.05  0.82

[9643 rows x 4 columns]


In [7]:
att_m2_fault10 = pd.read_csv("ATT.csv")
att_m2_fault10 = att_m2_fault10.astype("float64")
att_m2_fault10 = att_m2_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m2_fault10 = att_m2_fault10[((att_m2_fault10['TimeUS'] >= min) & (att_m2_fault10['TimeUS'] <= max	))]
att_m2_fault10=att_m2_fault10.reset_index(drop=True)
print(att_m2_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      76601803.0    -0.12 -0.11      0.44   3.26    3.67  3.62
1      76612061.0    -0.14  0.25      0.45   3.25    3.67  3.58
2      76622292.0    -0.17 -0.02      0.47   3.32    3.67  3.55
3      76632106.0    -0.20 -0.35      0.47   3.39    3.67  3.58
4      76643046.0    -0.22 -0.04      0.47   3.35    3.67  3.60
...           ...      ...   ...       ...    ...     ...   ...
9638  182650515.0     1.82 -0.61     -2.13  -0.99    1.57  0.82
9639  182660867.0     1.82 -0.54     -2.15  -0.98    1.57  0.82
9640  182671317.0     1.81 -0.47     -2.17  -0.99    1.57  0.83
9641  182681862.0     1.79 -0.43     -2.19  -1.00    1.57  0.82
9642  182692387.0     1.77 -0.43     -2.21  -1.05    1.56  0.82

[9643 rows x 7 columns]


In [8]:
from numpy import empty
to_add=[]

if len(att_m2_fault10)>len(xkf1_m2_fault10):
    to_add=att_m2_fault10[len(xkf1_m2_fault10):]
    att_m2_fault10=att_m2_fault10[:len(xkf1_m2_fault10)]

if len(xkf1_m2_fault10)>len(att_m2_fault10):
    to_add=xkf1_m2_fault10[len(att_m2_fault10):]

for idx,i in enumerate(att_m2_fault10['Roll']):
    if(xkf1_m2_fault10['Roll'][idx] != i):
        att_m2_fault10['Roll'][idx] = ((att_m2_fault10['Roll'][idx] + xkf1_m2_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m2_fault10['Pitch']):
    if(xkf1_m2_fault10['Pitch'][idx] != i):
        att_m2_fault10['Pitch'][idx] = ((att_m2_fault10['Pitch'][idx] + xkf1_m2_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m2_fault10['Yaw']):
    if(xkf1_m2_fault10['Yaw'][idx] != i):
        att_m2_fault10['Yaw'][idx] = ((att_m2_fault10['Yaw'][idx] + xkf1_m2_fault10['Yaw'][idx])/2)

if not empty(to_add):
    att_m2_fault10=pd.concat([att_m2_fault10,to_add])

print(att_m2_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0      76601803.0    -0.12 -0.11      0.44   3.26    3.67  3.62
1      76612061.0    -0.14  0.25      0.45   3.25    3.67  3.58
2      76622292.0    -0.17 -0.02      0.47   3.32    3.67  3.55
3      76632106.0    -0.20 -0.35      0.47   3.39    3.67  3.58
4      76643046.0    -0.22 -0.04      0.47   3.35    3.67  3.60
...           ...      ...   ...       ...    ...     ...   ...
9638  182650515.0     1.82 -0.61     -2.13  -0.99    1.57  0.82
9639  182660867.0     1.82 -0.54     -2.15  -0.98    1.57  0.82
9640  182671317.0     1.81 -0.47     -2.17  -0.99    1.57  0.83
9641  182681862.0     1.79 -0.43     -2.19  -1.00    1.57  0.82
9642  182692387.0     1.77 -0.43     -2.21  -1.05    1.56  0.82

[9643 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m2_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m2_fault10 = esc_0_m2_fault10.astype("float64")
esc_0_m2_fault10 = esc_0_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m2_fault10 = esc_0_m2_fault10[((esc_0_m2_fault10['TimeUS'] >= min) & (esc_0_m2_fault10['TimeUS'] <= max))]
esc_0_m2_fault10=esc_0_m2_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m2_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m2_fault10 = esc_1_m2_fault10.astype("float64")
esc_1_m2_fault10 = esc_1_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m2_fault10 = esc_1_m2_fault10[((esc_1_m2_fault10['TimeUS'] >= min) & (esc_1_m2_fault10['TimeUS'] <= max))]
esc_1_m2_fault10=esc_1_m2_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m2_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m2_fault10 = esc_2_m2_fault10.astype("float64")
esc_2_m2_fault10 = esc_2_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m2_fault10 = esc_2_m2_fault10[((esc_2_m2_fault10['TimeUS'] >= min) & (esc_2_m2_fault10['TimeUS'] <= max))]
esc_2_m2_fault10=esc_2_m2_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m2_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m2_fault10 = esc_3_m2_fault10.astype("float64")
esc_3_m2_fault10 = esc_3_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m2_fault10 = esc_3_m2_fault10[((esc_3_m2_fault10['TimeUS'] >= min) & (esc_3_m2_fault10['TimeUS'] <= max))]
esc_3_m2_fault10=esc_3_m2_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m2_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m2_fault10 = esc_4_m2_fault10.astype("float64")
esc_4_m2_fault10 = esc_4_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m2_fault10 = esc_4_m2_fault10[((esc_4_m2_fault10['TimeUS'] >= min) & (esc_4_m2_fault10['TimeUS'] <= max))]
esc_4_m2_fault10=esc_4_m2_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m2_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m2_fault10 = esc_5_m2_fault10.astype("float64")
esc_5_m2_fault10 = esc_5_m2_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m2_fault10 = esc_5_m2_fault10[((esc_5_m2_fault10['TimeUS'] >= min) & (esc_5_m2_fault10['TimeUS'] <= max))]
esc_5_m2_fault10=esc_5_m2_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m2_fault10 = pd.read_csv("ESC_0.csv")\nesc_0_m2_fault10 = esc_0_m2_fault10.astype("float64")\nesc_0_m2_fault10 = esc_0_m2_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m2_fault10 = esc_0_m2_fault10[((esc_0_m2_fault10[\'TimeUS\'] >= min) & (esc_0_m2_fault10[\'TimeUS\'] <= max))]\nesc_0_m2_fault10=esc_0_m2_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m2_fault10 = pd.read_csv("ESC_1.csv")\nesc_1_m2_fault10 = esc_1_m2_fault10.astype("float64")\nesc_1_m2_fault10 = esc_1_m2_fault10.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m2_fault10 = esc_1_m2_fault10[((esc_1_m2_fault10[\'TimeUS\'] >= min) & (esc_1_m2_fault10[\'TimeUS\'] <= max))]\nesc_1_m2_fault10=esc_1_m2_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m2_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m2_fault10 = imu_0_m2_fault10.astype("float64")
imu_0_m2_fault10 = imu_0_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m2_fault10 = imu_0_m2_fault10[((imu_0_m2_fault10['TimeUS'] >= min) & (imu_0_m2_fault10['TimeUS'] <= max))]

imu_1_m2_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m2_fault10 = imu_1_m2_fault10.astype("float64")
imu_1_m2_fault10 = imu_1_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m2_fault10 = imu_1_m2_fault10[((imu_1_m2_fault10['TimeUS'] >= min) & (imu_1_m2_fault10['TimeUS'] <= max))]

imu_2_m2_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m2_fault10 = imu_2_m2_fault10.astype("float64")
imu_2_m2_fault10 = imu_2_m2_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m2_fault10 = imu_2_m2_fault10[((imu_2_m2_fault10['TimeUS'] >= min) & (imu_2_m2_fault10['TimeUS'] <= max))]

imu_m2_fault10 = pd.concat((imu_0_m2_fault10, imu_1_m2_fault10, imu_2_m2_fault10))
imu_m2_fault10=imu_m2_fault10.groupby(imu_m2_fault10.TimeUS, as_index=False).mean()

display(imu_m2_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76596898.0,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309
1,76599612.0,0.629721,-0.022170,-0.038268,0.697404,1.520396,-9.676227
2,76601568.0,0.734351,-0.198096,-0.042900,0.828583,0.903306,-9.486217
3,76604796.0,-0.051758,-0.113764,0.003014,0.711837,0.047788,-9.498156
4,76606627.0,-0.559301,0.035907,0.019815,0.574881,-0.004650,-9.492326
...,...,...,...,...,...,...,...
36636,182681634.0,0.085096,-0.008251,-0.013832,-0.205013,0.486183,-9.757404
36637,182684723.0,0.118494,-0.062658,0.006834,-0.134503,0.242348,-9.748803
36638,182686868.0,-0.020153,-0.089405,0.008694,-0.236891,-0.043895,-9.593975
36639,182689400.0,-0.083203,-0.084285,0.006058,-0.320326,-0.045759,-9.582758


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m2_fault10 = pd.merge_ordered(imu_m2_fault10,att_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m2_fault10 = pd.merge_ordered(m2_fault10, esc_0_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_1_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_2_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_3_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_4_m2_fault10)
m2_fault10 = pd.merge_ordered(m2_fault10, esc_5_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")'''
m2_fault10 = pd.merge_ordered(m2_fault10, rcou_m2_fault10)
m2_fault10=m2_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m2_fault10["TimeUS"] = m2_fault10["TimeUS"] - m2_fault10.iloc[0]["TimeUS"]
m2_fault10["TimeUS"] = m2_fault10["TimeUS"].astype(int)
m2_fault10["Guasto"] = guasto

In [13]:
last_cell = m2_fault10.iloc[-1, m2_fault10.columns.get_loc('TimeUS')]
print(last_cell)

106101465


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

          TimeUS
0              0
1           2857
2           5714
3           8571
4          11428
...          ...
37133  106088981
37134  106091838
37135  106094695
37136  106097552
37137  106100409

[37138 rows x 1 columns]


In [16]:
m2_fault10_final = pd.merge_ordered(m2_fault10,df)
m2_fault10_final=m2_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m2_fault10_final)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY      AccZ  \
0              0 -0.087496  0.129375  0.002737  0.410752  1.279124 -9.849309   
1           2857 -0.087496  0.129375  0.002737  0.410752  1.279124 -9.849309   
2           5218 -0.087496  0.129375  0.002737  0.410752  1.279124 -9.849309   
3           5714 -0.087496  0.129375  0.002737  0.410752  1.279124 -9.849309   
4           7932  0.629721 -0.022170 -0.038268  0.697404  1.520396 -9.676227   
...          ...       ...       ...       ...       ...       ...       ...   
91160  106097720 -0.083203 -0.084285  0.006058 -0.320326 -0.045759 -9.582758   
91161  106100409 -0.083203 -0.084285  0.006058 -0.320326 -0.045759 -9.582758   
91162  106100597 -0.018030 -0.053529  0.014931 -0.345331  0.090727 -9.770250   
91163  106100707 -0.018030 -0.053529  0.014931 -0.345331  0.090727 -9.770250   
91164  106101465 -0.018030 -0.053529  0.014931 -0.345331  0.090727 -9.770250   

       DesRoll  Roll  DesPitch  Pitch  

In [17]:
m2_fault10_final = m2_fault10_final[m2_fault10_final.TimeUS.isin(to_be)]

In [18]:
m2_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
1,2857,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
3,5714,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
5,8571,0.629721,-0.022170,-0.038268,0.697404,1.520396,-9.676227,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
8,11428,0.734351,-0.198096,-0.042900,0.828583,0.903306,-9.486217,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91151,106088981,0.027898,-0.011213,-0.022906,-0.237379,0.488266,-9.606852,1.81,-0.47,-2.17,-0.99,1.57,0.83,1471.0,1457.0,1527.0,1392.0,1392.0,1527.0,2.0
91155,106091838,0.085096,-0.008251,-0.013832,-0.205013,0.486183,-9.757404,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0
91157,106094695,0.118494,-0.062658,0.006834,-0.134503,0.242348,-9.748803,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0
91159,106097552,-0.020153,-0.089405,0.008694,-0.236891,-0.043895,-9.593975,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0


In [19]:
from datetime import timedelta
m2_fault10_final=m2_fault10_final.reset_index(drop=True)
m2_fault10_final['TimeUS'] = pd.to_datetime(m2_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m2_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
1,00:00:00.002857,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
2,00:00:00.005714,-0.087496,0.129375,0.002737,0.410752,1.279124,-9.849309,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
3,00:00:00.008571,0.629721,-0.022170,-0.038268,0.697404,1.520396,-9.676227,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
4,00:00:00.011428,0.734351,-0.198096,-0.042900,0.828583,0.903306,-9.486217,-0.12,-0.11,0.44,3.26,3.67,3.62,1355.0,1440.0,1435.0,1362.0,1393.0,1405.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37133,00:01:46.088981,0.027898,-0.011213,-0.022906,-0.237379,0.488266,-9.606852,1.81,-0.47,-2.17,-0.99,1.57,0.83,1471.0,1457.0,1527.0,1392.0,1392.0,1527.0,2.0
37134,00:01:46.091838,0.085096,-0.008251,-0.013832,-0.205013,0.486183,-9.757404,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0
37135,00:01:46.094695,0.118494,-0.062658,0.006834,-0.134503,0.242348,-9.748803,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0
37136,00:01:46.097552,-0.020153,-0.089405,0.008694,-0.236891,-0.043895,-9.593975,1.79,-0.43,-2.19,-1.00,1.57,0.82,1496.0,1379.0,1460.0,1421.0,1365.0,1508.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m2_fault10_final)/350)):
        V1=m2_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_15296\138999135.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_15296\138999135.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,0.031982,0.048433,3.204854,0.222076,65.0,83.0,10.918404,7.603945,-0.051246,0.040877,-0.463970,0.208294,0.0,91.0,39.398149,2.167955,0.159693,0.020620,-1.721277,0.214622,72.0,60.0,1.861878,0.637409,0.074637,0.113768,-0.743482,0.344985,0.0,93.0,38.182008,22.212815,0.368697,0.132007,1.619943,0.517271,105.0,108.0,19.367219,15.951396,-10.393854,0.638644,-0.906896,10.424444,0.0,121.0,163.904047,13.844325,0.108604,0.542982,-0.599546,0.743794,0.0,90.0,130.062101,2.881083,0.971738,4.702028,-1.696500,2.373374,122.0,121.0,4.880665,4.762715,6.023020,8.786648,-0.624521,6.711064,103.0,76.0,-1.989107,-2.405632,1550.720798,7398.727538,0.685374,1553.097756,91.0,89.0,3681.634842,1298.182046,1502.239316,4765.313993,-0.568462,1503.820036,100.0,99.0,2661.472567,2095.376153,1552.185185,3472.568466,-0.574423,1553.300204,131.0,129.0,1263.213875,1110.461140,1503.948718,4766.888791,-0.298633,1505.528165,130.0,131.0,1062.533118,794.066709,1496.920228,4471.605047,-1.145481,1498.408834,101.0,100.0,4062.897777,1401.730852,1557.521368,5077.067399,-0.174105,1559.145732,100.0,102.0,1724.164654,1394.258289,2
1,1.0,0.031864,0.048300,3.221266,0.221761,65.0,84.0,12.629785,7.645930,-0.050853,0.040815,-0.459907,0.208050,0.0,66.0,39.322680,3.263814,0.160073,0.020612,-1.720928,0.214888,58.0,44.0,1.878713,0.532791,0.074882,0.113465,-0.737921,0.344602,0.0,88.0,38.606416,22.497364,0.369306,0.131762,1.626302,0.517469,104.0,110.0,21.327926,15.604353,-10.393160,0.636994,-0.899566,10.423673,0.0,122.0,163.342784,13.978941,0.111335,0.544061,-0.620604,0.744924,0.0,87.0,130.196240,2.754213,0.970000,4.689695,-1.693350,2.370078,130.0,129.0,17.675595,5.232748,6.042585,8.896361,-0.631732,6.736759,105.0,78.0,-2.005043,-2.118865,1550.928977,7392.903773,0.693665,1553.303767,101.0,98.0,3730.987497,1194.596933,1502.093750,4759.196314,-0.568218,1503.672607,98.0,97.0,2733.883997,2287.491982,1552.423295,3482.632276,-0.571208,1553.541382,125.0,123.0,1239.562055,1138.372699,1503.750000,4767.207977,-0.307569,1505.329774,124.0,125.0,1175.112657,807.658435,1496.818182,4462.530951,-1.144179,1498.303882,109.0,108.0,4046.725696,1239.139459,1557.687500,5072.318020,-0.168324,1559.310186,108.0,110.0,1699.673899,1394.278266,2
2,2.0,0.031830,0.048163,3.238767,0.221449,65.0,83.0,14.571096,7.709035,-0.050875,0.040699,-0.452846,0.207779,0.0,63.0,39.097796,3.398926,0.160444,0.020602,-1.720451,0.215142,72.0,51.0,1.893569,0.548559,0.074995,0.113147,-0.731658,0.344167,0.0,93.0,38.980319,22.622216,0.369233,0.131389,1.639016,0.517059,102.0,108.0,22.048532,15.601450,-10.392759,0.635241,-0.892516,10.423190,0.0,129.0,162.668640,13.920076,0.114051,0.545119,-0.641215,0.746045,0.0,80.0,130.310580,2.616750,0.968272,4.677426,-1.690192,2.366797,138.0,137.0,17.605281,5.572431,6.062040,9.00468

In [23]:
path_file = path_file.replace(r"M2\FAULT_M2_10", "")
os.chdir(path_file)
df_merged.to_csv('m2_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
